# kcBERT QA를 이용하여 형량 예측

## 1. 패키지 로드

In [1]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
import tensorflow as tf
import tqdm.notebook
import math
import urllib.request
import json

## 2. Base Model & Tokenizer load

In [2]:
model = TFAutoModelForQuestionAnswering.from_pretrained('beomi/kcbert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Pipeline 구성

In [3]:
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

## 4. Fine-tuning
- korquad(한국어 QA 데이터 셋)을 이용한 파인튜닝([링크](https://korquad.github.io/))

### 1) korquad 데이터 다운로드

In [4]:
urllib.request.urlretrieve('https://korquad.github.io/dataset/KorQuAD_v1.0_train.json', 'korquad.json')
korquad = json.load(open('korquad.json', encoding='utf8'))

### 2) Input 데이터 전처리 함수 정의

In [6]:
def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def make_inputs(context, qas):
    inputs = tokenizer(
        context, 
        qas['question'], 
        truncation=True, 
        max_length=tokenizer.model_max_length)
    q = qas['answers'][0]
    start_char = q['answer_start']
    end_char = start_char + len(q['text']) - 1
    start = inputs.char_to_token(0, start_char)
    end = inputs.char_to_token(0, end_char)
    inputs['start_positions'] = [start]
    inputs['end_positions'] = [end]
    return inputs

### 3) Input 데이터 전처리

In [7]:
%%time
n = 0
filename = 'korquad.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
    for item in korquad['data']:   # 각 아이템 순환
        for para in item['paragraphs']:  # 아이템마다 
            context = para['context']
            for qas in para['qas']:
                inputs = make_inputs(context, qas)
                if inputs['start_positions'][0] and inputs['end_positions'][0]:
                    feature = {k: int_feature(v) for k, v in inputs.items()}
                    example = tf.train.Example(features=tf.train.Features(feature=feature))
                    s = example.SerializeToString()
                    writer.write(s)
                    n += 1

Wall time: 29.6 s


In [8]:
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
int_value = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'start_positions': int_value,
    'end_positions': int_value
}

In [9]:
def preproc(example):
    example = tf.io.parse_single_example(example, feature_description)
    example = {k : tf.cast(v, tf.int32) for k, v in example.items()}
    return example

In [10]:
# korquad 데이터셋 불러오기
dataset = tf.data.TFRecordDataset(['korquad.tfrecord']).map(preproc).padded_batch(8)
batch = next(iter(dataset))
result = model(batch)

### 4) Train

In [11]:
%%time
# 옵티마이저는 adam
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)

n = 57653
for batch in tqdm.notebook.tqdm(dataset, total=math.ceil(n / 32)):
    with tf.GradientTape() as tape:
        result = model(batch)
        loss = tf.reduce_mean(result['loss'])
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables)) # loss가 감소하는 방향으로 파라미터 업데이트

  0%|          | 0/1802 [00:00<?, ?it/s]

Wall time: 38min 8s


## 5. Test

### 1) QA function 정의

In [12]:
def qa(question, context):
    inputs = tokenizer(context, question, add_special_tokens=True, return_tensors="tf")
    outputs = model(inputs)
    start = tf.argmax(outputs.start_logits, axis=1).numpy()[0]
    end = tf.argmax(outputs.end_logits, axis=1).numpy()[0]
    return tokenizer.decode(inputs['input_ids'][0, start:end+1])

### 2) Simple data test

In [13]:
context = "피고에게 징역 62년의 형량을 선고한다."

In [14]:
qa("형량은 얼마인가?", context)

'징역 62년'

### 3) complex data test

In [15]:
context = """원심판결을 파기한다.

피고인을 금고 4월에 처한다.
1. 항소이유의 요지

원심이 선고한 형(금고 1년에 집행유예 2년)에 대하여 피고인은 너무 무거워서, 검사는 너무 가벼워서 부당하다고 각각 주장한다.

2. 판 단

피고인이 이 사건 범행을 인정하면서 잘못을 반성하고 있고, 피해자 ●●●의 유족 및 피해자 ◎◎◎과 각 합의하여 이들이 피고인의 처벌을 원치 않는 점, 피고인에게 소년보호처분 이외에 전과가 없는 점은 인정된다.

그러나 이 사건 범행은 피고인이 제한속도가 시속 80km인 도로를 시속 200km가 넘는 속도로 과속운전을 하다가 발생한 것으로 피고인의 주의의무 위반의 정도가 매우 크고, 이 사건으로 인하여 피해 차량의 운전자가 사망하고, 가해 차량의 동승자가 중상을 입는 중대한 결과가 초래된 점에서, 피고인에게는 그에 상응하는 처벌이 필요하다.

위와 같은 사정들과 그 밖에 피고인의 성행, 환경, 범행 후의 정황 등 기록과 변론에 나타난 모든 양형조건을 종합하여 보면, 원심이 선고한 형은 너무 가벼워서 부당하다.

3. 결 론

그렇다면 검사의 항소는 이유 있으므로 형사소송법 제364조 제6항에 따라 원심판결을 파기하고, 변론을 거쳐 다시 다음과 같이 판결한다(피고인의 항소는 이유 없으나, 검사의 항소를 받아들여 원심판결을 파기하는 이상 따로 주문에서 피고인의 항소를 기각하지 아니한다).

범죄사실 및 증거의 요지

이 법원이 인정하는 범죄사실 및 증거의 요지는 원심판결 각 해당란 기재와 같으므로 형사소송법 제369조에 따라 그대로 인용한다.

법령의 적용

1. 범죄사실에 대한 해당법조

교통사고처리 특례법 제3조 제1항, 형법 제268조(업무상과실치사의 점), 교통사고처리 특례법 제3조 제1항, 제2항 단서 제3호, 형법 제268조(업무상과실치상의 점)

1. 상상적 경합

형법 제40조, 제50조[죄질이 더 무거운 교통사고처리특례법위반(치사)죄에 정한 형으로 처벌]

1. 형의 선택

금고형 선택
"""

In [20]:
qa("최종판결의 형량은 무엇인가?", context)

'2년'

- 기계독해를 이용하여 형량을 집계한 후 중앙값, 평균, 최빈값의 형식으로 예상형량을 보여준다.